In [5]:
import pandas as pd
import google.generativeai as genai, os, time
import json, pathlib, re, math, random
from typing import List, Dict
from dataclasses import dataclass
from tqdm import tqdm

with open('../env.json', 'r') as f:
    env_vars = json.load(f)
    
os.environ["GOOGLE_API_KEY"] = env_vars["GOOGLE_API_KEY"]

#env.json
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-2.0-flash")


# model.generate_content = retry.Retry(predicate=lambda e: hasattr(e, 'code') and e.code in {429, 503})(model.generate_content)


# df = pd.read_csv("../data/FineTunedResponses/responses_to_fine_tuned.csv")

In [5]:
rubric_instructions = '''
You are an expert evaluator for AI-generated content.
Rate the following response according to this enhanced rubric:

- Relevance (1-5): How directly the response addresses the specific query or task.
- Accuracy (1-5): Factual correctness and absence of errors or misleading information.
- Completeness (1-5): Coverage of all required aspects of the task, with appropriate depth.
- Clarity (1-5): Organization, coherence, and ease of understanding.
- Creativity (1-5): Originality of approach, uniqueness of insights, and innovative thinking.
- Conciseness (1-5): Efficiency of expression without unnecessary verbosity.
- Technical Correctness (1-5): For technical responses, accuracy of code, formulas, or technical concepts.
- Actionability (1-5): How immediately useful and implementable the response is.

Additionally, rate the Task Category (select one):
- Informational (seeking factual information)
- Creative (requiring original content creation)
- Technical (code, technical explanations)
- Problem-solving (solutions to specific problems)
- Analytical (evaluation or analysis of information)

Provide the scores as a JSON object with a weighted overall score:
{
  "task_category": "Technical",
  "relevance": 5,
  "accuracy": 4,
  "completeness": 5,
  "clarity": 4,
  "creativity": 3,
  "conciseness": 4,
  "technical_correctness": 5,
  "actionability": 5,
  "weighted_score": 4.6
}

The weighted_score should be calculated based on task category:
- Informational: relevance*0.25 + accuracy*0.3 + completeness*0.2 + clarity*0.15 + conciseness*0.1
- Creative: relevance*0.2 + completeness*0.15 + clarity*0.15 + creativity*0.4 + conciseness*0.1
- Technical: relevance*0.15 + accuracy*0.2 + completeness*0.15 + clarity*0.15 + technical_correctness*0.25 + actionability*0.1
- Problem-solving: relevance*0.2 + accuracy*0.15 + completeness*0.15 + clarity*0.1 + actionability*0.4
- Analytical: relevance*0.2 + accuracy*0.25 + completeness*0.2 + clarity*0.15 + creativity*0.2

Then briefly justify your scores.
'''

In [6]:
def evaluate_with_gemini(prompt, response, expected):
    full_input = f"""
{rubric_instructions}

Prompt:
{prompt}

Expected Answer:
{expected}

Model Response:
{response}
"""
    result = model.generate_content(full_input)
    return result.text

# For programmatic access to scores in your analysis:
def extract_scores(evaluation_text):
    import json
    import re
    
    # Find JSON object in the evaluation text
    json_pattern = r'\{[\s\S]*?\}'
    match = re.search(json_pattern, evaluation_text)
    
    if match:
        try:
            scores = json.loads(match.group())
            return scores
        except json.JSONDecodeError:
            return None
    return None

In [6]:
# Create an subsection copy of the sample data
df_sample = df.head(3).copy()

# Use loc (I had to look this up) to set the values
df_sample.loc[:, "evaluation"] = df_sample.apply(
    lambda row: evaluate_with_gemini(
        row["original_prompt"],
        row["response_to_fine_tuned"],
        row["fine_tuned_prompt"]  # Using fine_tuned_prompt as the expected answer
    ), axis=1
)

NameError: name 'df' is not defined

In [9]:
#Prints the subsection to check the values and make sure it looks right
print(df_sample[["evaluation"]])

                                          evaluation
0  ```json\n{\n  "task_category": "Analytical",\n...
1  ```json\n{\n  "task_category": "Analytical",\n...
2  ```json\n{\n  "task_category": "Technical",\n ...


In [31]:
from chromadb import Documents, EmbeddingFunction, Embeddings
import google.generativeai as genai
from google.api_core import retry
from google.api_core import exceptions

# A function that uses Gemini model to convert the responses into embeddings.
# Can generate embeddings for documents or queries depending on if the variable document_mode is set to true or false.
class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True  # Specify whether to generate embeddings for documents or queries

    def is_retriable(e):
        return isinstance(e, exceptions.ServiceUnavailable) or \
               isinstance(e, exceptions.ResourceExhausted) or \
               (hasattr(e, 'code') and e.code in {429, 503})

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        embedding_task = "retrieval_document" if self.document_mode else "retrieval_query"

        if isinstance(input, str):
            input = [input]
        
        embeddings = []
        for text in input:
            response = genai.embed_content(
                model="models/embedding-001",
                content=text,
                task_type=embedding_task
            )
            # Extract just the embedding values from the response (With the call of the embedding row)
            embeddings.append(response['embedding'])
        
        return embeddings

In [43]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from chromadb import Documents, Embeddings

# A class that uses the rubric and the embeddings to grade the responses given to it
class RubricGrader:
    def __init__(self):
        self.embedding_function = GeminiEmbeddingFunction()
        # Define rubric categories (Im not sure if we should have this as a dictionary or a list, and also if we should have categories for grading as well or just a single rubric)
        # Like we will have to find a way to get a rubric for each model, and then get it so tthat we take the model that has the top graded response 
        self.rubric_prompts = {
            "Excellent": "The submission is well-organized, addresses the prompt thoroughly, and provides in-depth analysis with robust supporting evidence.",
            "Good": "The submission addresses most aspects of the prompt with clear points but may lack depth in analysis.",
            "Average": "The submission responds to the prompt in a basic manner, though the analysis and organization are limited.",
            "Poor": "The submission is incomplete, lacks coherence, and does not satisfactorily address the prompt."
        }
        # Precompute embeddings for rubric descriptions
        self.rubric_embeddings = self._compute_rubric_embeddings()
    
    def _compute_rubric_embeddings(self):
        descriptions = list(self.rubric_prompts.values())
        # Get embeddings for all descriptions at once
        embeddings = self.embedding_function(descriptions)
        return {grade: emb for grade, emb in zip(self.rubric_prompts.keys(), embeddings)}
    
    def grade_submission(self, submission_prompt: str):
        # Get embedding for the submission
        submission_embedding = self.embedding_function([submission_prompt])[0]
        
        best_grade = None
        highest_similarity = -1
        
        # Compare against each rubric embedding
        for grade, rubric_embedding in self.rubric_embeddings.items():
            sim = cosine_similarity([submission_embedding], [rubric_embedding])[0][0]
            if sim > highest_similarity:
                highest_similarity = sim
                best_grade = grade
                
        return best_grade, highest_similarity

# Call the grader
grader = RubricGrader()

C:\Users\ianra\AppData\Local\Temp\ipykernel_30136\3316484650.py:8: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  self.embedding_function = GeminiEmbeddingFunction()


### The cell below evaluates prompts using a rubric-based grading system with embeddings generated by Gemini. The system works by:

1. **Rubric Definition**
   - Defines 4 grade levels: Excellent, Good, Average, and Poor
   - Each grade has a detailed description of what constitutes that level of quality
   - These descriptions serve as reference points for grading

2. **Embedding Generation**
   - Uses Google's Gemini model to convert both:
     - The rubric descriptions into numerical vectors (embeddings)
     - Each prompt into its own embedding

3. **Similarity Scoring**
   - Compares each prompt's embedding to the rubric embeddings using cosine similarity
   - Scores range from 0 (completely different) to 1 (identical)
   - The grade with the highest similarity score is assigned to the prompt

4. **Batch Processing**
   - Processes prompts in batches of 10 to manage memory and API usage
   - Includes error handling for invalid or empty prompts
   - Saves progress periodically to prevent data loss

### Understanding the Results:

The output includes:
- Grade distribution showing how many prompts fall into each category
- Average similarity scores for each grade level
- Sample graded prompts with their scores
- Statistics about processed and skipped prompts

#### Interpreting Similarity Scores
- Higher scores (closer to 1.0) indicate stronger matches with the rubric criteria
- Scores typically fall between 0.6 and 0.8 (Can be much better with more training)
- Small differences in scores can be meaningful due to the high-dimensional nature of embeddings

#### Limitations/What needs improvement
- The system relies on semantic similarity, which may not capture all aspects of quality
- Very short or very long prompts may be graded differently due to noise in the text (Preprocessing hopefully fixes but no guarantee)
- The quality of grading depends on how well the rubric descriptions differentiate between grades, should be well but may have to train it better still
- Need to get a way to follow similar strategy with all models for future comparison, without it taking too long
   - End goal is to be able to have grading good enough to where each model has qualities its best at and can clearly be chosen

In [42]:
import pandas as pd
from tqdm import tqdm
import time
import numpy as np

# Read the CSV file
df = pd.read_csv('../data/ShareGPT/separated_prompts_clean.csv').head(100)

# Initialize the grader
grader = RubricGrader()

# Create lists to store results
results = []

# Define batch size
BATCH_SIZE = 10
SLEEP_TIME = .25  # seconds

# Process prompts in batches
for start_idx in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing batches"):
    batch_df = df.iloc[start_idx:start_idx + BATCH_SIZE]
    batch_results = []
    
    # Process each prompt in the current batch
    for idx, row in batch_df.iterrows():
        prompt = row['prompt']
        
        # Skip NaN or empty prompts
        if pd.isna(prompt) or not isinstance(prompt, str):
            print(f"Skipping prompt {row['original_id']}: Invalid prompt value")
            continue
            
        try:
            grade, similarity = grader.grade_submission(prompt)
            batch_results.append({
                'original_id': row['original_id'],
                'prompt': prompt,
                'grade': grade,
                'similarity_score': similarity
            })
        except Exception as e:
            print(f"Error processing prompt {row['original_id']}: {str(e)}")
    
    # Add batch results to main results list
    results.extend(batch_results)
    
    # Save intermediate results after each batch
    if len(results) % (BATCH_SIZE * 10) == 0:  # Save every 10 batches
        interim_df = pd.DataFrame(results)
        interim_df.to_csv(f'prompt_grading_results_interim_{len(results)}.csv', index=False)
    
    # Sleep between batches to avoid rate limits
    time.sleep(SLEEP_TIME)

# Convert all results to DataFrame
results_df = pd.DataFrame(results)

# Display summary statistics
print("\nGrade Distribution:")
print(results_df['grade'].value_counts())

print("\nAverage Similarity Score by Grade:")
print(results_df.groupby('grade')['similarity_score'].mean())

# Save final results
results_df.to_csv('prompt_grading_results_final.csv', index=False)
print("\nFinal results saved to prompt_grading_results_final.csv")

# Display sample of results
print("\nSample of graded prompts:")
print(results_df.sample(5))

# Print statistics about skipped prompts
print("\nTotal prompts processed:", len(results))
print("Total prompts in dataset:", len(df))
print("Skipped prompts:", len(df) - len(results))

C:\Users\ianra\AppData\Local\Temp\ipykernel_30136\458185774.py:7: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  self.embedding_function = GeminiEmbeddingFunction()
Processing batches: 100%|██████████| 10/10 [00:20<00:00,  2.01s/it]


Grade Distribution:
grade
Poor         60
Good         16
Excellent    14
Average      10
Name: count, dtype: int64

Average Similarity Score by Grade:
grade
Average      0.709245
Excellent    0.687306
Good         0.754996
Poor         0.737981
Name: similarity_score, dtype: float32

Final results saved to prompt_grading_results_final.csv

Sample of graded prompts:
    original_id                                             prompt      grade  \
94        89757  Now, you need to act both as an interviewer an...  Excellent   
35        16382  give me a terraform module that will run sever...       Poor   
26        54555   Could you write cucumber scenarios for such app?       Good   
60       175882       Create a good improv comedy script on crypto       Poor   
6         73273  I have a complex task for you\r\n\r\nSeparate ...       Poor   

    similarity_score  
94          0.658958  
35          0.671703  
26          0.779992  
60          0.784713  
6           0.802017  

Tota

## Analyzing responses for best attributs in certain categories

In [46]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class ResponseEvaluator:
    def __init__(self):
        self.embedding_function = GeminiEmbeddingFunction()
        
        # Define rubric descriptions for each category and level
        self.category_rubrics = {
            'Relevance': {
                5: "The response directly and fully addresses all aspects of the prompt with perfect alignment",
                4: "The response addresses the main aspects of the prompt with good alignment",
                3: "The response somewhat addresses the prompt but may have some tangential content",
                2: "The response only partially addresses the prompt with significant deviation",
                1: "The response barely addresses or misses the point of the prompt entirely"
            },
            'Accuracy': {
                5: "The response is completely factual with precise, verifiable information",
                4: "The response is mostly accurate with minor imprecisions",
                3: "The response has a mix of accurate and questionable information",
                2: "The response contains several inaccuracies",
                1: "The response is largely incorrect or misleading"
            },
            'Completeness': {
                5: "The response comprehensively covers all required elements with additional valuable insights",
                4: "The response covers all required elements thoroughly",
                3: "The response covers most required elements with some gaps",
                2: "The response misses several required elements",
                1: "The response is significantly incomplete"
            },
            'Clarity': {
                5: "The response is exceptionally well-structured, clear, and easy to understand",
                4: "The response is well-organized and clearly expressed",
                3: "The response is generally clear but could be better organized",
                2: "The response is somewhat unclear or poorly structured",
                1: "The response is confusing and poorly organized"
            }
        }
        
        # Pre-compute embeddings for all rubric descriptions
        self.category_embeddings = self._compute_rubric_embeddings()
    
    def _compute_rubric_embeddings(self):
        embeddings = {}
        for category, levels in self.category_rubrics.items():
            category_emb = {}
            for level, description in levels.items():
                emb = self.embedding_function([description])[0]
                category_emb[level] = emb
            embeddings[category] = category_emb
        return embeddings
    
    def evaluate_response(self, response: str):
        # Get embedding for the response
        response_embedding = self.embedding_function([response])[0]
        
        # Evaluate each category
        scores = {}
        similarities = {}
        for category, level_embeddings in self.category_embeddings.items():
            # Compare with each level in the category
            level_scores = {}
            for level, level_emb in level_embeddings.items():
                sim = cosine_similarity([response_embedding], [level_emb])[0][0]
                level_scores[level] = sim
            
            # Get the best matching level
            best_level = max(level_scores.items(), key=lambda x: x[1])
            scores[category] = best_level[0]
            similarities[category] = best_level[1]
        
        return {
            'scores': scores,
            'similarities': similarities,
            'overall_score': np.mean(list(scores.values())),
            'strongest_category': max(similarities.items(), key=lambda x: x[1])[0],
            'weakest_category': min(similarities.items(), key=lambda x: x[1])[0]
        }

def evaluate_responses(responses: list, prompts: list = None):
    """
    Evaluate multiple responses and return the best one
    """
    evaluator = ResponseEvaluator()
    results = []
    
    for i, response in enumerate(responses):
        result = evaluator.evaluate_response(response)
        result['response'] = response
        result['prompt'] = prompts[i] if prompts else None
        results.append(result)
    
    # Sort by overall score
    results.sort(key=lambda x: x['overall_score'], reverse=True)
    
    return {
        'best_response': results[0],
        'all_results': results,
        'summary': {
            'average_score': np.mean([r['overall_score'] for r in results]),
            'score_distribution': {
                category: np.mean([r['scores'][category] for r in results])
                for category in results[0]['scores'].keys()
            }
        }
    }


In [53]:
import pandas as pd
import time

# Read the CSV file
df = pd.read_csv('../data/FineTunedResponses/responses_to_fine_tuned.csv').head(100)

# Define batch parameters
BATCH_SIZE = 20
SLEEP_TIME = .2  # seconds between batches

# Initialize empty list for all results
all_batch_results = []

# Process in batches
for start_idx in range(0, len(df), BATCH_SIZE):
    print(f"\nProcessing batch {start_idx//BATCH_SIZE + 1} of {(len(df) + BATCH_SIZE - 1)//BATCH_SIZE}")
    
    # Get current batch
    batch_df = df.iloc[start_idx:start_idx + BATCH_SIZE]
    
    # Prepare responses and prompts for this batch
    batch_responses = batch_df['response_to_fine_tuned'].tolist()
    batch_prompts = batch_df['original_prompt'].tolist()
    
    try:
        # Evaluate the batch
        batch_results = evaluate_responses(batch_responses, batch_prompts)
        all_batch_results.append(batch_results)
        
        # Print batch summary
        print(f"Batch best response score: {batch_results['best_response']['overall_score']:.2f}")
        print(f"Batch average score: {batch_results['summary']['average_score']:.2f}")
        
        # Save intermediate results
        batch_results_df = pd.DataFrame([{
            'original_id': batch_df.iloc[i]['original_id'],
            'original_prompt': batch_df.iloc[i]['original_prompt'],
            'response': result['response'],
            'overall_score': result['overall_score'],
            **{f'score_{category}': score for category, score in result['scores'].items()},
            'strongest_category': result['strongest_category'],
            'weakest_category': result['weakest_category']
        } for i, result in enumerate(batch_results['all_results'])])
        
       #batch_results_df.to_csv(f'response_evaluation_results_batch_{start_idx//BATCH_SIZE + 1}.csv', index=False)
        
    except Exception as e:
        print(f"Error processing batch {start_idx//BATCH_SIZE + 1}: {str(e)}")
    
    # Sleep between batches
    time.sleep(SLEEP_TIME)

# Combine all results
all_results = []
all_responses = []
for batch_result in all_batch_results:
    all_results.extend(batch_result['all_results'])
    
# Calculate overall statistics
overall_results = {
    'best_response': max(all_results, key=lambda x: x['overall_score']),
    'all_results': all_results,
    'summary': {
        'average_score': sum(r['overall_score'] for r in all_results) / len(all_results),
        'score_distribution': {
            category: sum(r['scores'][category] for r in all_results) / len(all_results)
            for category in all_results[0]['scores'].keys()
        }
    }
}

# Print final results
print("\n=== Final Results ===")
print("\nBest Overall Response:")
print(f"Overall Score: {overall_results['best_response']['overall_score']:.2f}")
print("\nCategory Scores:")
for category, score in overall_results['best_response']['scores'].items():
    print(f"{category}: {score}")

print("\nOverall Summary:")
print(f"Average Score: {overall_results['summary']['average_score']:.2f}")
print("\nCategory Averages:")
for category, score in overall_results['summary']['score_distribution'].items():
    print(f"{category}: {score:.2f}")

# Save final combined results
final_results_df = pd.DataFrame([{
    'original_id': df.iloc[all_results.index(result)]['original_id'],
    'original_prompt': df.iloc[all_results.index(result)]['original_prompt'],
    'response': result['response'],
    'overall_score': result['overall_score'],
    **{f'score_{category}': score for category, score in result['scores'].items()},
    'strongest_category': result['strongest_category'],
    'weakest_category': result['weakest_category']
} for result in all_results])

final_results_df.to_csv('response_evaluation_results_final.csv', index=False)
print("\nFinal results saved to response_evaluation_results_final.csv")


Processing batch 1 of 5


C:\Users\ianra\AppData\Local\Temp\ipykernel_30136\694891130.py:6: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  self.embedding_function = GeminiEmbeddingFunction()


Batch best response score: 5.00
Batch average score: 3.99

Processing batch 2 of 5


C:\Users\ianra\AppData\Local\Temp\ipykernel_30136\694891130.py:6: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  self.embedding_function = GeminiEmbeddingFunction()


Batch best response score: 5.00
Batch average score: 3.86

Processing batch 3 of 5


...

Batch best response score: 5.00
Batch average score: 4.20

Processing batch 4 of 5


C:\Users\ianra\AppData\Local\Temp\ipykernel_30136\694891130.py:6: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  self.embedding_function = GeminiEmbeddingFunction()


Batch best response score: 4.75
Batch average score: 3.88

Processing batch 5 of 5


C:\Users\ianra\AppData\Local\Temp\ipykernel_30136\694891130.py:6: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  self.embedding_function = GeminiEmbeddingFunction()


Batch best response score: 5.00
Batch average score: 4.05

=== Final Results ===

Best Overall Response:
Overall Score: 5.00

Category Scores:
Relevance: 5
Accuracy: 5
Completeness: 5
Clarity: 5

Overall Summary:
Average Score: 4.00

Category Averages:
Relevance: 3.25
Accuracy: 4.46
Completeness: 3.41
Clarity: 4.86

Final results saved to response_evaluation_results_final.csv
